# Deep residual network（ResNet）

[Introduction about ResNet](https://zhuanlan.zhihu.com/p/31852747)

[ResNet Wiki](https://en.wikipedia.org/wiki/Residual_neural_network)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D

import skimage
from skimage.transform import rescale, resize

import pydot

In [2]:
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

print('Importing successfully!')
print('tensorflow',tf.__version__)
print('tensorflow.keras',tf.keras.__version__)
print('keras',keras.__version__)
print('GPU',tf.config.list_physical_devices('GPU'))

Importing successfully!
tensorflow 2.3.0
tensorflow.keras 2.4.0
keras 2.4.3
GPU [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Parameter setting


In [3]:
BS = 128
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 170

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

Resize_pixelsize = 197

# Build Model

In [4]:
vgg_notop = VGGFace(model='resnet50', include_top=False, input_shape=(Resize_pixelsize, Resize_pixelsize, 3), pooling='avg')

last_layer = vgg_notop.get_layer('avg_pool').output

x = Flatten(name='flatten')(last_layer)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc7')(x)
x = Dropout(DROPOUT_RATE)(x)

In [5]:
batch_norm_indices = [2, 6, 9, 13, 14, 18, 21, 24, 28, 31, 34, 38, 41, 45, 46, 53, 56, 60, 63, 66, 70, 73, 76, 80, 83, 87, 88, 92, 95, 98, 102, 105, 108, 112, 115, 118, 122, 125, 128, 132, 135, 138, 142, 145, 149, 150, 154, 157, 160, 164, 167, 170]
for i in range(FROZEN_LAYER_NUM):
    if i not in batch_norm_indices:
        vgg_notop.layers[i].trainable = False

In [6]:
out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(vgg_notop.input, out)

In [7]:
optim = keras.optimizers.Adam(lr=ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optim = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = keras.optimizers.SGD(lr=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# ImageData Generator

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(dataset, aug=False):
    if aug:
        datagen = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
            dataset,
            target_size=(197, 197),
            color_mode='rgb',
            shuffle = True,
            class_mode='categorical',
            batch_size=BS)

In [9]:
train_generator  = get_datagen("/usr/users/gpupro/gprcsr1_1/Desktop/ST7_FER_Projet/FER_Dataset/train", True)
dev_generator    = get_datagen("/usr/users/gpupro/gprcsr1_1/Desktop/ST7_FER_Projet/FER_Dataset/test-private")
test_generator  = get_datagen("/usr/users/gpupro/gprcsr1_1/Desktop/ST7_FER_Projet/FER_Dataset/test-public")

Found 29903 images belonging to 7 classes.
Found 3590 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


# Train the Model

In [ ]:
EPOCHS = 100

history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=28709// BS,
    validation_steps=3509 // BS,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=[rlrop],
    use_multiprocessing=False,
) 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
224/224 [==============================] - 207s 925ms/step - loss: 1.8511 - accuracy: 0.3487 - val_loss: 1.9045 - val_accuracy: 0.1638
Epoch 2/100
224/224 [==============================] - 248s 1s/step - loss: 1.3639 - accuracy: 0.4854 - val_loss: 2.0066 - val_accuracy: 0.1780
Epoch 3/100
224/224 [==============================] - 203s 906ms/step - loss: 1.2615 - accuracy: 0.5245 - val_loss: 1.4588 - val_accuracy: 0.4413
Epoch 4/100
224/224 [==============================] - 218s 972ms/step - loss: 1.1919 - accuracy: 0.5523 - val_loss: 1.0969 - val_accuracy: 0.5964
Epoch 5/100
224/224 [==============================] - 198s 886ms/step - loss: 1.1504 - accuracy: 0.5693 - val_loss: 1.0323 - val_accuracy: 0.6218
Epoch 6/100
224/224 [==============================] - 203s 906ms/step - loss: 1.1116 - accuracy: 0.5839 - val_loss: 1.0006 - val_accuracy: 0.6319
Epoch 7/100
224/224 [========================

# Evaluation

In [ ]:
print('\n# Evaluate on dev data')
results_dev = model.evaluate_generator(dev_generator, 3509 // BS)
print('dev loss, dev acc:', results_dev)

In [ ]:
print('\n# Evaluate on test data')
results_test = model.evaluate_generator(test_generator, 3509 // BS)
print('test loss, test acc:', results_test)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

# Save the Model

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
test_acc = 'test_acc_%.3f' % results_test[1]
model.save('trained_models/' + 'RESNET50' + epoch_str + test_acc + '.h5')